In [4]:
import pandas as pd
import sqlite3
import re
import numpy as np
import time

In [2]:
import spacy
import multiprocessing
from tqdm import tqdm

c:\Users\dell\anaconda3\envs\cse446\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Start of cleaning the review dataset
conn = sqlite3.connect("yelp_data.db")
review_df = pd.read_sql(
    "SELECT * FROM review", conn)
conn.close()

review_df.tail()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date,review_score
4988729,7NgXAuTFiJHYbuepOPwU0w,x1QLCwZGFAjxRRw4EHc3-g,1_BVWDzi5cVqWxNe9bOMMQ,5.0,1,0,1,Don't misinterpret my 5-star review....I don't...,2016-04-30 01:02:34,0.8
4988730,wD5ZWao_vjyT2h4xmGam8Q,7L7GL5Pi2cf8mbm2Dpw4zw,e_E-jq9mwm7wk75k7Yi-Xw,5.0,1,0,1,It is very rare for a restaurant to be this go...,2022-01-17 22:36:01,0.8
4988731,YVX1Wsa4LYxjvFwuHBb_gA,RKPkxOYQlM0BjhM-H6_vAw,X4mouE_cMiwbfyCPZ_K-FA,4.0,3,0,2,"Good, maybe very good. I went for lunch, so t...",2015-01-23 23:52:03,2.2
4988732,i-I4ZOhoX70Nw5H0FwrQUA,YwAMC-jvZ1fvEUum6QkEkw,Rr9kKArrMhSLVE9a53q-aA,5.0,1,0,0,For when I'm feeling like ignoring my calorie-...,2022-01-19 18:59:27,0.6
4988733,RwcKOdEuLRHNJe4M9-qpqg,6JehEvdoCvZPJ_XIxnzIIw,VAeEXLbEcI9Emt9KGYq9aA,3.0,10,3,7,Located in the 'Walking District' in Nashville...,2018-01-02 22:50:47,8.0


In [6]:
from nltk.tokenize import TreebankWordTokenizer
from nltk.corpus import stopwords
import swifter

# Initialize the tokenizer
tokenizer = TreebankWordTokenizer()

def preprocess_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'[^\w\s]', '', text)  # Remove special characters
    words = tokenizer.tokenize(text)  # Use TreebankWordTokenizer instead of word_tokenize
    words = [word for word in words if word not in stopwords.words('english')]  # Remove stopwords
    return " ".join(words)

sample_df = review_df.sample(10000).copy()

# Time swifter apply
start = time.time()
sample_df["cleaned_text"] = sample_df["text"].swifter.apply(preprocess_text)
end = time.time()
print(f"Swifter processed 10,000 rows in {end - start:.2f} seconds")

# # Apply preprocessing to the DataFrame
# review_df["cleaned_text"] = review_df["text"].swifter.apply(preprocess_text)

# # Display the first few rows
# review_df.head()


Pandas Apply:  26%|██▌       | 2607/10000 [01:06<03:07, 39.33it/s]


KeyboardInterrupt: 

In [ ]:
import time
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])

def spacy_clean_doc(doc):
    return " ".join([token.text for token in doc if token.is_alpha and not token.is_stop])

# Get a test batch of 100,000 reviews
texts = review_df["text"].astype(str).iloc[:100000].tolist()

# Track time
start = time.time()

cleaned = []
for doc in tqdm(nlp.pipe(texts, batch_size=1000, n_process=4), total=len(texts)):
    cleaned.append(spacy_clean_doc(doc))

end = time.time()
print(f"\n Completed 1 batch of 100,000 reviews in {end - start:.2f} seconds")

  0%|          | 0/100000 [00:00<?, ?it/s]

In [ ]:
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])

def spacy_clean_doc(doc):
    return " ".join([token.text for token in doc if token.is_alpha and not token.is_stop])

texts = review_df["text"].astype(str).tolist()

batch_size = 100000
all_cleaned = []

for i in range(0, len(texts), batch_size):
    batch = texts[i:i+batch_size]
    cleaned = []

    print(f"\n Processing batch {i} to {i+batch_size}...")

    for doc in tqdm(nlp.pipe(batch, batch_size=1000, n_process=1), total=len(batch)):
        cleaned.append(spacy_clean_doc(doc))

    # Save partial result if needed
    batch_df = pd.DataFrame({"text": batch, "cleaned_text": cleaned})
    batch_df.to_csv(f"cleaned_batch_{i}.csv", index=False)

    all_cleaned.extend(cleaned)

review_df["cleaned_text"] = all_cleaned

100%|██████████| 5000/5000 [00:30<00:00, 166.12it/s]


ValueError: Length of values (5000) does not match length of index (4988734)